In [1]:
!pip install -e ..

Obtaining file:///export/home/fornax/jhoskins/Development/astrohack
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for astrohack (pyproject.toml) ... done
  Created wheel for astrohack: filename=astrohack-0.0.1-py3-none-any.whl size=1052 sha256=a0dfcf04762da4eaca02f7c5a63dd8a41561b1bd664405f288d01df397757744
  Stored in directory: /tmp/pip-ephem-wheel-cache-gtzthqxj/wheels/92/c4/f7/67427f82ce804bcc66fc63663cafee2be9a98c9f78ddf37022
Successfully built astrohack
  Attempting uninstall: astrohack
    Found existing installation: astrohack 0.0.1
    Uninstalling astrohack-0.0.1:
      Successfully uninstalled astrohack-0.0.1


In [2]:
import os
import tarfile
import shutil
import dask
import requests
import time
import psutil

import numpy as np
import matplotlib.pyplot as plt

from astrohack import holog
from astrohack.dio import extract_holog
from astrohack.dio import load_hack_file
from astrohack._utils._io import _make_ant_pnt_dict

from dask.distributed import Client, LocalCluster

plt.rcParams["figure.figsize"] = (15, 15)

In [3]:
def performance(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        stop = time.time()
        print("Completion time: {}".format(stop - start))
    return wrapper

def print_hack_file(hack):
    for ddi in hack.keys():
        for scan in hack[ddi].keys():
            for ant in hack[ddi][scan].keys():
                print("="*150)
                print("\t\t\t\t\t\t\tddi={ddi}\tscan={scan}\t ant={ant}".format(ddi=ddi, scan =scan, ant=ant))
                print("="*150)
                print("{xds}".format(xds=hack[ddi][scan][ant]))

def get_max_memory_allocation():
    memory_string = str(int((psutil.virtual_memory().total/1e9)/os.cpu_count()))
    return "".join((memory_string, 'GB'))

def get_example_data():
    google_file_id = '1zpOtduyXtbh0wg0s5KWYuEMy7roGq92a'
    output = 'alma_band3.calibrated.DV16.ms.tar'
    path = "/".join((os.getcwd(), output))
    
    download_file_from_google_drive(
        id=google_file_id,
        destination=path
    )

    
    tar = tarfile.open(path)
    tar.extractall()
    tar.close()
    
    os.remove(path)
    
    return path.split('.tar')[0]

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id , 'confirm': 1 }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [4]:
ms_name = get_example_data()
#ms_name = '/users/jhoskins/fornax/Development/J1924-2914.ms.split.calibrated.SPW3'

In [5]:
try:
    client = Client('http://127.0.0.1:8787')

except:
    cluster = LocalCluster(
        n_workers=6,
        threads_per_worker=1,
        memory_limit='8GB'
    )

    client = Client(cluster)
        
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [6]:
DA = ['DA41', 'DA42', 'DA43', 'DA44', 'DA45', 'DA46', 'DA48', 'DA49', 'DA50', 'DA51', 'DA52', 'DA53', 'DA54', 'DA55', 'DA56', 'DA57', 'DA58', 'DA59']
DV = ['DV02', 'DV03', 'DV04', 'DV11', 'DV12', 'DV13', 'DV14', 'DV15', 'DV16', 'DV17', 'DV18', 'DV19', 'DV20', 'DV21', 'DV22', 'DV23', 'DV24', 'DV25'] 

holog_obs_description = {
    0:{
        2:{
            'map':DV,
            'ref':DA
        } 
    }
}

#holog_obs_description = {0:{2:{'map':DV,'ref':DA}, 6:{'map':DV,'ref':DA}, 10:{'map':DV,'ref':DA},4:{'map':DA,'ref':DV}, 8:{'map':DA,'ref':DV}, 12:{'map':DA,'ref':DV}}} 

In [7]:
extract_holog(
    ms_name=ms_name, 
    hack_name='hack.dict', 
    holog_obs_dict=holog_obs_description,
    data_col='DATA',
    subscan_intent='MIXED',
    parallel=False
)

client.get_events("runtimes")

Successful readonly open of usernoread-locked table /export/home/fornax/jhoskins/Development/astrohack/examples/alma_band3.calibrated.DV16.ms/ANTENNA: 8 columns, 36 rows
/export/home/fornax/jhoskins/Development/astrohack/examples/alma_band3.calibrated.DV16.ms/DATA_DESCRIPTION
Processing ddi: 0, scan: 2
dict_keys(['ms_name', 'hack_name', 'pnt_name', 'ddi', 'data_col', 'chan_setup', 'pol_setup', 'map_ant_ids', 'map_ant_names', 'ref_ant_ids', 'sel_state_ids', 'scan'])
vis data type  complex64
Successful readonly open of default-locked table /export/home/fornax/jhoskins/Development/astrohack/examples/alma_band3.calibrated.DV16.ms/ANTENNA: 8 columns, 36 rows
In scan  2  antenna  18  is flagged
In scan  2  antenna  19  is flagged
In scan  2  antenna  20  is flagged
In scan  2  antenna  21  is flagged
In scan  2  antenna  22  is flagged
In scan  2  antenna  23  is flagged
In scan  2  antenna  24  is flagged
In scan  2  antenna  25  is flagged
Done


()

In [8]:
holog(hack_name='hack.dict')

No workers found


In [10]:
load_hack_file('hack.dict', load_pnt_dict=False)
#hack.keys()

{0: {2: {32: <xarray.Dataset>
   Dimensions:              (time: 2401, lm: 2, chan: 64, pol: 4)
   Coordinates:
     * chan                 (chan) float64 1.09e+11 1.09e+11 ... 1.109e+11 1.11e+11
     * pol                  (pol) int32 9 10 11 12
     * time                 (time) float64 5.045e+09 5.045e+09 ... 5.045e+09
   Dimensions without coordinates: lm
   Data variables:
       DIRECTIONAL_COSINES  (time, lm) float64 dask.array<chunksize=(2401, 2), meta=np.ndarray>
       VIS                  (time, chan, pol) complex64 dask.array<chunksize=(1201, 32, 2), meta=np.ndarray>
       WEIGHT               (time, chan, pol) complex64 dask.array<chunksize=(1201, 32, 2), meta=np.ndarray>
   Attributes:
       ant_id:               32
       ddi:                  0
       parallactic_samples:  [4.299527027776883, 4.3015339559019825, 4.303337694...
       scan:                 2,
   28: <xarray.Dataset>
   Dimensions:              (time: 2401, lm: 2, chan: 64, pol: 4)
   Coordinates:
     

In [ ]:
def _read_data_from_hack_meta(ant):
    
    ant = str(ant)

    file = "hack.dict/.hack_json"

    hack = load_hack_file('hack.dict', load_pnt_dict=False)

    with open(file, "r") as json_file: 
        hack_json = json.load(json_file)
    
    ant_sub_dict = {}
    ant_data_dict = {}
    
    for ddi in hack_json[ant].keys():
        for scan in hack_json[ant][ddi].keys():
            ant_data_dict.setdefault(int(ddi), {})[int(scan)]= hack[int(ddi)][int(scan)][int(ant)]
            
    return ant_data_dict
        
        
_read_data_from_hack_meta(32)

In [ ]:
print_hack_file(hack)            

In [ ]:
time = hack[0][2][26].time.values
l = hack[0][2][26].DIRECTIONAL_COSINES[:, 0]
m = hack[0][2][26].DIRECTIONAL_COSINES[:, 1]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1)

ax[0].scatter(l, m)
ax[0].set_xlabel('DIRECTIONAL COSINE(L)')
ax[0].set_ylabel('DIRECTIONAL COSINE(M)')

ax[1].scatter(time, l, label='L')
ax[1].scatter(time, m, label='M')

ax[1].set_xlabel('time')
ax[1].set_ylabel('DIRECTIONAL COSINES(L, M)')

ax[1].legend(bbox_to_anchor=(1.02, 1.0), loc='upper left', borderaxespad=0.)

plt.show()